In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

schema = StructType([
    StructField("patient_id", IntegerType()),
    StructField("timestamp", StringType()),
    StructField("heart_rate", IntegerType()),
    StructField("blood_pressure", StringType()),
    StructField("oxygen_saturation", IntegerType())
])

# Read simulated stream from TCP socket
raw_df = (spark.readStream
    .format("socket")
    .option("host", "localhost")
    .option("port", 9999)
    .load())

# Parse JSON lines
parsed_df = (raw_df
    .select(from_json(col("value"), schema).alias("data"))
    .select("data.*"))

# Write to console (or Delta)
(parsed_df.writeStream
    .outputMode("append")
    .format("console")
    .option("truncate", False)
    .trigger(processingTime="5 seconds")  # every 5 seconds
    .start())




Out[14]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fa6c2b28a60>